# **Packages & Libraries**

## Hazm

In [ ]:
pip install hazm

## Import & Download hazm models

In [ ]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download('punkt')
import hazm
import re
import pandas as ps

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
!gdown 1rr7j4FHIumIFmqgVsQHbhU8Q0vv7t1TR

Downloading...
From: https://drive.google.com/uc?id=1rr7j4FHIumIFmqgVsQHbhU8Q0vv7t1TR
To: /content/pos_tagger.model
100% 19.2M/19.2M [00:00<00:00, 47.7MB/s]


In [ ]:
import gdown
gdown.download_folder('https://drive.google.com/drive/folders/1Zjg-rYwSrk4pXwB1Lm3Q6IfHvyxeJspX?usp=drive_link', quiet=True)

['/content/universal_dependency_parser/.DS_Store',
 '/content/universal_dependency_parser/langModel.mco',
 '/content/universal_dependency_parser/lib/liblinear-1.8.jar',
 '/content/universal_dependency_parser/lib/libsvm.jar',
 '/content/universal_dependency_parser/lib/log4j.jar',
 '/content/universal_dependency_parser/malt-features.xml',
 '/content/universal_dependency_parser/malt-options.xml',
 '/content/universal_dependency_parser/malt.jar',
 '/content/universal_dependency_parser/MaltEval.jar']

## Stop words

In [ ]:
!wget -O stopwords.txt https://raw.githubusercontent.com/kharazi/persian-stopwords/master/short

--2024-05-19 18:51:13--  https://raw.githubusercontent.com/kharazi/persian-stopwords/master/short
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2945 (2.9K) [text/plain]
Saving to: ‘stopwords.txt’

stopwords.txt       100%[===================>]   2.88K  --.-KB/s    in 0s      

2024-05-19 18:51:13 (29.9 MB/s) - ‘stopwords.txt’ saved [2945/2945]



In [ ]:
# /content/augmented_data.txt
# !wget https://raw.githubusercontent.com/language-ml/parsi.io/main/parsi_io/modules/product_feature_extractor/data/quality_pre_words.txt
# !wget https://github.com/language-ml/parsi.io/blob/main/parsi_io/modules/product_feature_extractor/data/augmented_data.txt
!git clone --depth=1 https://github.com/language-ml/parsi.io.git

Cloning into 'parsi.io'...
remote: Enumerating objects: 245, done.
remote: Counting objects: 100% (245/245), done.
remote: Compressing objects: 100% (213/213), done.
remote: Total 245 (delta 14), reused 215 (delta 14), pack-reused 0
Receiving objects: 100% (245/245), 42.12 MiB | 15.22 MiB/s, done.
Resolving deltas: 100% (14/14), done.
Updating files: 100% (217/217), done.


# **Sentence tokenizer**

In [ ]:
comments = [
    'بد نیست کار راه اندازه ',
    'با سلام انتخاب خوبی بود به جا',
    'طبق ادعای دیجی کالا نات اکتیو بود و از لحاظ قیمتی تو شگفت انگیز خوب بود',
    'تازه چند روزه بدستم رسیده تا الان که خوب بوده راضیم',
    'گوشی آیفون نیاز به تعریف نداره ، منکه راضی بودم بیشتر دنبال zaa بودم اما دی جی کالا ch موجود داشت نحوه بسته بندی و ارسال عالی',
    'ایفون که نیاز به تعریف نداره عالیه',
    'خوبه من راضی بودم همه چیز مرتب و بدون ایراد',
    'قیمت مناسب نسبت به بازار و نات اکتیو',
    'خوب وعالی ومناسب بودممنونم',
    'خوبه در کل راضی کننده اس ',
    'واقعا عالی بود ممنونم از دیجی کالا',
    'همه چی عالی بود',
    'من سفارش دادم وقتی بسته رسید دیدم پلمپش باز شده بود که اصلا کار قشنگی نیست برای دیجی کالا و فروشنده',
    'باز نشده بود تمیز و سالم بدستم رسید',
    'عالی بود سالم و پلمپ به دستم رسیر',
    'گوشی خیلی خوبی هست فقط باید حواستون باشه اپل آیدی رو گم نکنید رنگ سبزش خیلی خوش رنگ تر از بقیه رنگ ها هست لوازم جانبی هرچی نیاز دارین از دیجی کالا بگیرید چون بیرون قیمت ها خیلی بالایه',
    'عالی نات اکیتو همون روز خرید تحویل شد',
    'من توی شگفتانه خریدم تقریبا دو تومن زیر قیمت بازار گرفتم ریجستری هم شد مشکلی نداشت',
    'بسیار عالی و باکیفیت نیازی به تعریف نیست',
    '۴_۵ساعته برام ارسال شد و گوشی نات اکتیو بودن و ریجستر شد',
    'دو هفته دیرتر تحویل دادند.ریجستریش مشکل داشت،کلی وقتم برای یه گوشی تلف شد',
    'عالی بود و بر خلاف یسری از نظرات گوشی اصل و پلمپ و نات اکتیو بود و به راحتی ریجستر شد همچنین سالم ممنون از دیجی کالا',
    'پیشنهاد میکنم حتما خرید کنید و امیدوارم مثل الماس تو دستتون بدرخشه،👌👍',
    'فوق العاده ست اپل',
    'تو گوشیهای آیفون موجود و ریجستر شده به روزترینه و نیاز به کاربر معمولی رو برآورده می کنه.',
    'امروز رسید دستم خیلی عالی زیبا خوش دست حس خیلی خوبی رو به آدم منتقل میکنه خصوصا اگه مثل من خیلی وقته ذوق داشتنشو دارید ظاهر رنگ سفید استارلایتش فوق العاده جذابه من بین این رنگ و صورتی مونده بودم و خوشحالم این رنگ و انتخاب کردم یکم واسه بحث رجیستریش میترسیدم واسه همین خیلی مردد بودم از کدوم فروشنده بگیرم گارانتیش اوکی باشه . پایاگستر و انتخاب کردم با گارانتی ارتباط ژیوان قبلش سرچ کردم سایتشون اوکی بود و پاسخگوی مشتری هاشون بودن. و خداروشکر رجیستر شد . شماره گارانتی رو توی سا ...',
    'مشکل فیس آیدی داشت اصلأ از دیجی‌کالا گوشی نخرید با خیال راحت برید حضوری خرید کنید',
    'خرید این گوشی را با توچه به دوربین عالی و با کیفیتش ، سی پی یو قدرتمندش، باتری و شارژدهی مناسبش همچنین نمایشگر باکیفیت و طراحی زیبای آن پیشنهاد میکنم',
    'ممنون بابت وقت شناسی برای ارسال دقیق و به موقع و البته بسته بندی مناسب و همچنین پلمپ و سالم بودن دستگاه',
    'ایفون دیگه نیازی به چیزی نداره نقاط قوت و ضعفش معلومه فقط اینجا میتونستم نحوه رسیدن و مدت رسیدن و بسته بندی رو بگم که اونا هم عالی بودن تهران بودم شب خریدم فرداش رسید دستم',
    'با خیال راحت از دیجی کالا بخرید (خودم با استرس گرفتم ولی خیلی زود رسید و همه چیز خوب بود نات بود بسته بندی مناسب در کل خوب بود همه چیز خرید کنید لذت ببرید)',
    'گوشی خوبی هست اما متأسفانه تک‌سیمکارته هست، با اینکه تو اطلاعات فروشش نوشته بود دو‌سیمکارته',
    'خرید تو یک روز ‌و تحویل هم در همان روز واقعا عالی و بی نقص ریجستری سریع٫خیلی گوشی خوش دست و عالی هستش',
    'لطفا گوشی را در بسته بندی محفوظ تری ارسال کنید . تولید ۲۰۲۴ بود بسته بندی گوشی کاملا پلمپ و سالم ارسال شد',
    'دلار گرون میشه به ثانیه نمیکشه گرون میکنن حالا که ارزون شده عین خیالشونم نیست',
    'کد فعالسازی رو که وارد میکنم میگه نامعتبر و رجیستر نمیشه',
    'به موقع به دستم رسید.من برای همسرم هدیه گرفتم رنگ سبز تیره که خیلی قشنگه. هنوز روشنش نکردیم اما همه چیز اوکی بود ظاهرا&amp;#34;. قیمتش نسبت به بازار ارزونتر بود.',
    'همه چیز،عالی بسته بندی ارسال بموقع ممنون از دبجی کالا ممنون از دیجی پی',
    'آیفون نیاز به تعریف نداره فقط دوستان بعد از زدن اپل آیدی حتما find my IPhone خاموش کنین تا اپل آیدیتون لاک نشه من از دیجی پی اعتبار گرفتم خیلی خوبه زود اعتبار داد که تونستم بخرم ممنون از دیجی پی😍',
    'از همه نظر چک کردم اوکی و نات اکتیو بود ممنون از دیجی....',
    'ایفون دیگه نیاز به تعریف نداره ممنون از دیجی بابت ارسال به موقع کالا 🩷👍',
    'خیلی راضی ام از خرید این گوشی با تشکر از دیجی کالا',
    'سفیدشم تخفیف بذارید لطفااااا',
    'به توجه به اینکه سری های جدیدتر این گوشی رجیستر نمیشه و ممکنه از بیرون نتونین پک اصلی رو با اطمینان بخرید، اینجا با اطمینان به مهلت مرجوعی میتونین خریدتون رو انجام بدید. درضمن برای خرید گوشی آیفون که رجیستر بشه، گزینه ی دیگه ای هم نیست که بخواین بهش فکر کنین و همین تنها گزینه موجوده. پس تا گرونتر نشده یا موجودی پک اصلی تمام نشده، خریدتون رو انجام بدید. درمورد خود گوشی هم حرفی برای گفتن نیست و گوشی حرف نداره!',
    'در دیجی کالا قیمت خیلی مناسبتر است',
    'به شخصه با این طراحی ایفون ۱۳ بیشتر حال میکنم تا ۱۵ متاسفانه به خودم دیجی پی اعتبار نمیداد با حساب پدرم گرفتم! کاش میشد اعتبار رو انتقال داد تا بتونم حداقل با حساب کاربری خودم بگیرم از این فروشنده هم راضیم مشکلی تو کار نبود',
    'از نظر من گوشی خوبه‌ایه ولی ارزش خرید نسبت به این قیمت رو نمی‌دونم داره یا نه مثلا ویدیو های افراد رو ببینید و بعد خرید کنید یه عالم ویژگی خوب داره ها ولی زود داغ میکنه ',
    'بعد از شش ماه استفاده و رعایت نکاتی که نوشته باید بهش توجه بشه ، سلامت باتری به ۹۱ درصد رسیده و به شدت همیشه موقع استفاده داغ میکنه واقعا نمیدونم ایفون قیمت بالایی ندارد',
    'سلام ممنونم از دیجی کالا این سومین گوشی بود ک خریداری کردم از دیجی کالا ولی متاسفانه اینبار مشخصات ناقص بود گوشی یک سیمکارت زد بودن دو سیمکارت',
    'من از طریق اعتبار دیجی پی خریدم و واقعا واسم بصرفه تر از خرید قسطی از مغازه بود٫ بسته بندی مشکلی نداشت و به موقع بدستم رسید ٫مشخصات تو سایت چک کردم و سری اصلی بود٫ من رنگ مشکی خریدم ولی رنگ سفید پیشنهاد میکنم و خودم برگردم عقب سفید میگیرم!',

    'گوشی واقعا راضی کننده است. باطری گوشی یک روز پر کار رو همراهی میکنه و تو نیم ساعت 50 درصدش پر میشه دوربین فوق العاده ای داره و راضیتون میکنه پردازنده اش اصلا هنگ نمیکنه و داغ نمیشه و برنامه های سنگین رو راحت اجرا میکنه صفحه نمایشش با 900 نیت روشنایی کاملا تصاویر رو با کیفیت و واضع نشون میده. ظاهر کوشی هم خیلی شیک و خوشگله. رابط کاربری روان و حذابی داره. در کل گوشی قابل قبولی است و قیمتش نسبت به مشخصاتش خیلی ارزونه.'
]

In [ ]:
normalizer = hazm.Normalizer()
lemmatizer = hazm.Lemmatizer()
stemmer = hazm.Stemmer()
tagger = hazm.POSTagger(model='pos_tagger.model')
parser = hazm.DependencyParser(tagger = tagger, lemmatizer = lemmatizer)

In [ ]:
def persian_sentence_tokenizer(text):
  # Define the regex pattern for sentence splitting
  conjunction_words = r"\s+(?:و|اما|ولی|ولیکن)\s+"
  pattern = f"{conjunction_words}|[.؟!]+"

  # Split the text using the pattern
  sentences = re.split(pattern, text)

  # Remove empty sentences
  sentences = [sentence.strip() for sentence in sentences if sentence.strip()]

  return sentences


def split_sentence_on_verb(sentence):
    # Tokenize and tag the sentence
    tokens = word_tokenize(sentence)
    tagged_tokens = tagger.tag(tokens)

    sentences = []
    current_sentence = []

    for word, tag in tagged_tokens:
        current_sentence.append(word)
        if tag == 'VERB':
            sentences.append(' '.join(current_sentence))
            current_sentence = []

    # Append the last accumulated sentence if any
    if current_sentence:
        sentences.append(' '.join(current_sentence))

    return sentences


def persian_sentence_tokenizer_v3(text):
    def contains_verbs(tags):
        counter = 0
        for tag in tags:
            if tag in ['VERB']:
                counter += 1
                if counter>1:
                  return True
        return False

    def contains_noun_or_ez(tagged_tokens):
        for word, tag in tagged_tokens:
            if 'NOUN' in tag:
                return True
        return False


    tagger = hazm.POSTagger(model = 'pos_tagger.model')
    sentences = []
    for sentence in sent_tokenize(text):
        tokens = word_tokenize(sentence)
        tagged_tokens = tagger.tag(tokens)
        tags = [tag for _, tag in tagged_tokens]
        if contains_verbs(tags):
            temp_sentences = split_sentence_on_verb(sentence)
            for temp_sentence in temp_sentences:
                temp_tokens = word_tokenize(temp_sentence)
                temp_tagged_tokens = tagger.tag(temp_tokens)
                if contains_noun_or_ez(temp_tagged_tokens):
                    # print(temp_tagged_tokens)
                    sentences.append(temp_sentence)
                else:
                    if sentences[-1]:
                        sentences[-1] += (" و "+temp_sentence)
                    else:
                        sentences.append(temp_sentence)
        else:
          sentences.append(sentence)

    return sentences

In [ ]:
# Example usage
sentence = sent_tokenize(comments[50])[1]
split_sentences = split_sentence_on_verb(sentence)
print(split_sentences)

['باطری گوشی یک روز پر کار رو همراهی میکنه', 'و تو نیم ساعت 50 درصدش پر میشه دوربین فوق العاده ای داره', 'و راضیتون میکنه', 'پردازنده اش اصلا هنگ نمیکنه', 'و داغ نمیشه', 'و برنامه های سنگین رو راحت اجرا میکنه', 'صفحه نمایشش با 900 نیت روشنایی کاملا تصاویر رو با کیفیت و واضع نشون میده', '.']


In [ ]:
tokenized_sent = persian_sentence_tokenizer(comments[50])
tokenized_sent1 = persian_sentence_tokenizer_v3(comments[50])

In [ ]:
tokenized_sent

['گوشی واقعا راضی کننده است',
 'باطری گوشی یک روز پر کار رو همراهی میکنه',
 'تو نیم ساعت 50 درصدش پر میشه دوربین فوق العاده ای داره',
 'راضیتون میکنه پردازنده اش اصلا هنگ نمیکنه',
 'داغ نمیشه',
 'برنامه های سنگین رو راحت اجرا میکنه صفحه نمایشش با 900 نیت روشنایی کاملا تصاویر رو با کیفیت',
 'واضع نشون میده',
 'ظاهر کوشی هم خیلی شیک',
 'خوشگله',
 'رابط کاربری روان',
 'حذابی داره',
 'در کل گوشی قابل قبولی است',
 'قیمتش نسبت به مشخصاتش خیلی ارزونه']

In [ ]:
tokenized_sent1

['گوشی واقعا راضی کننده است.',
 'باطری گوشی یک روز پر کار رو همراهی میکنه',
 'و تو نیم ساعت 50 درصدش پر میشه دوربین فوق العاده ای داره',
 'و راضیتون میکنه',
 'پردازنده اش اصلا هنگ نمیکنه و و داغ نمیشه',
 'و برنامه های سنگین رو راحت اجرا میکنه',
 'صفحه نمایشش با 900 نیت روشنایی کاملا تصاویر رو با کیفیت و واضع نشون میده و .',
 'ظاهر کوشی هم خیلی شیک و خوشگله.',
 'رابط کاربری روان و حذابی داره.',
 'در کل گوشی قابل قبولی است و قیمتش نسبت به مشخصاتش خیلی ارزونه.']

# **SmartPhoneFeatureExtractor Class**

In [ ]:
import re

def extract_battery_sentences(sent):
    # Keywords related to smartphone battery in Persian
    battery_keywords = r'(باتری|باطری|شارژ|شارژدهی|شارژ سریع|شارژ بی‌سیم|شارژر|پاوربانک)'

    # Regex pattern to match sentences containing battery-related keywords
    pattern = r'\b(' + battery_keywords + r')\b'

    return re.search(pattern, sent) is None

# Example usage
persian_comments = """
گوشی خیلی خوبی است. باتری آن بسیار قوی است و مدت زمان زیادی شارژ نگه می‌دارد.
اما در مقایسه با گوشی‌های دیگر، عمر باتری کمتری دارد.
شارژر سریع همراه دستگاه بسیار مفید است.
"""

extracted_sentences = extract_battery_sentences(tokenized_sent[1])


In [ ]:
print(extracted_sentences)

False


Please explain by example the functionality of these functions and what are they doing? Explain completely and step by step

In [ ]:
data_path = '/content/parsi.io/parsi_io/modules/product_feature_extractor/data'

class MyNormalizer(hazm.Normalizer):

    def __init__(self):
        super().__init__()
        self.aug_data_init()

    def aug_data_init(self):
        f = open(f'{data_path}/augmented_data.txt', 'r')
        self.aug_data = f.readlines()
        f.close()

    def replace_with_persian(self, sent):
        replace_list = {'ى':'ی',
                        'ة':'ه',
                        'ي':'ی'}

        for x in replace_list:
            sent = sent.replace(x, replace_list[x])

        return sent


    def my_normalize(self, text):
        text = self.normalize(text)

        aug = {y[0]:y[1] for y in [x.replace('\n', '').split('\t') for x in self.aug_data]}

        for k in aug:
            while k in text:
                text = text.replace(k, aug[k])
        while '\u200c' in text:
            text = text.replace('\u200c', ' ')
        return text

In [ ]:
class SmartPhoneFeatureExtractor():
    def __init__(self):
        self.normalizer = MyNormalizer()

    def persian_sentence_tokenizer(self, text):
      # Define the regex pattern for sentence splitting
      conjunction_words = r"\s+(?:و|اما|ولی|ولیکن)\s+"
      pattern = f"{conjunction_words}|[.؟!]+"

      # Split the text using the pattern
      sentences = re.split(pattern, text)

      # Remove empty sentences
      sentences = [sentence.strip() for sentence in sentences if sentence.strip()]

      return sentences


    def sent_normalize(self, sent):
        return self.normalizer.my_normalize(sent)


    def clean_result(self, text):
        for i in range(len(text)):
            for base_form in self.quality_base_forms:
                text[i] = re.sub(fr"({base_form[0]})", f"{base_form[1]}", text[i])
        return text


    def find_words(self, sent, pattern):
        matchs = []
        for m in re.finditer(pattern, sent):
            text = m.group()
            sp = list(m.span())
            if text[0] == ' ':
                sp[0] += 1
                text = text[1:]
            if text[-1] == ' ':
                sp[-1] -= 1
                text = text[:-1]
            matchs.append([text, sp])
        return matchs

    def extract_value(self, d: dict):
        result = None
        for key, value in d.items():
            if key != 'not' and value is not None:
                result = key
            if key == 'not' and value is not None:
                result = self.size_not_mapper[result]
        return result


    def general_opinion(self, text):

        noun_phrases = [
            r'بنظرم',
            r'به نظر من',
            r'به نظر میرسه',
            r'بنظر',
            r'به نظر',
            r'به نسبت',
            r'نسبت  به بقیه',
            r'همه چیز',
            r'همه‌چیز',
            r'در کل',
            r'واقعا',
            r'نسبت  به بقیه',
            r'از همه نظر',
            r'کاملا',
        ]

        demonstrative_adj = [
            r'این',
            r'آن',
            r'اون',
            r'ان',
        ]
        device_keywords = r"\b(گوشی|تلفن همراه|تلفن|موبایل)\b"

        device_postfix = [
            r'ا',
            r'\u200cها',
            # r'ا',
            # r'ا',
            # r'ا',
        ]

        pos_adj = [
            r'خیلی خوب',
            r'خیلی عالی',
            r'عالی',
            r'فوق العاده',
            r'بهتر از انتظارم',
            r'بی‌نظیر',
            r'خوب',
            r'راضی کننده',
            r'مناسب',
            r'درجه یک',
        ]

        pos_sent = [
            r'راضی بود',
            r'رضایت داشت',
            # r'',
        ]
        verb_identifier1 = r"ه|یه"

        verb_identifier2 = [
            r"(م|\u200cم|)",
            r"(ند|\u200cاند)",
        ]

        neg_adj = [
            r'خیلی بد',
            r'بد',
            r'بد',
            r'افتضاح',
            r'ناامید کننده',
            r'ضعیف',
            r'متوسط',
            r'معمولی',
            r'نامناسب',
        ]

        adj_postfix = [
            r'ی',
            r'ای',
            r' ای',
            r'\u200cای'

        ]

        pos_verb = [
            r'بود',
            r'هست',
            r'است',
        ]
        neg_verb = [
            r'نبود',
            r'نیست',
        ]

        # ساخت پترن های نظرات مثبت
        pos_pattern1 = rf"(?:{'|'.join(noun_phrases)})?{demonstrative_adj}*{device_keywords}{device_postfix}*.*?\b({'|'.join(pos_adj)})(?:{'|'.join(adj_postfix)})?\b.*?\b({'|'.join(pos_verb)})\b"
        pos_pattern2 = rf"(?:{'|'.join(noun_phrases)})?{demonstrative_adj}.*?\b({'|'.join(pos_adj)})(?:{'|'.join(adj_postfix)})?\b.*?\b({'|'.join(pos_verb)})\b"
        pos_pattern3 = rf"(?:{'|'.join(noun_phrases)})?.*?\b({'|'.join(pos_adj)})(?:{'|'.join(adj_postfix)})?\b.*?\b({'|'.join(pos_verb)})\b"
        pos_pattern4 = rf"{'|'.join(noun_phrases)}.*({'|'.join(pos_adj)})(?:{verb_identifier1})?"
        pos_pattern5 = rf"(?:{'|'.join(noun_phrases)})?.*{device_keywords}{device_postfix}*.*?\b({'|'.join(pos_sent)})(?:{'|'.join(verb_identifier2)})?\b.*"

        # ساخت پترن های نظرات منفی
        neg_pattern1 = rf"(?:{'|'.join(noun_phrases)})?{demonstrative_adj}*{device_keywords}{device_postfix}*.*?\b({'|'.join(neg_adj)})(?:{'|'.join(adj_postfix)})?\b.*?\b({'|'.join(pos_verb)})\b"
        neg_pattern2 = rf"(?:{'|'.join(noun_phrases)})?{demonstrative_adj}.*?\b({'|'.join(neg_adj)})(?:{'|'.join(adj_postfix)})?\b.*?\b({'|'.join(pos_verb)})\b"
        neg_pattern3 = rf"(?:{'|'.join(noun_phrases)})?.*?\b({'|'.join(neg_adj)})(?:{'|'.join(adj_postfix)})?\b.*?\b({'|'.join(pos_verb)})\b"
        neg_pattern4 = rf"{'|'.join(noun_phrases)}.*({'|'.join(neg_adj)})(?:{verb_identifier1})?"
        neg_pattern5 = rf"(?:{'|'.join(noun_phrases)})?{device_keywords}.*?\b({'|'.join(pos_adj)})(?:{'|'.join(adj_postfix)})?\b.*?\b({'|'.join(neg_verb)})\b"
        neg_pattern6 = rf"(?:{'|'.join(noun_phrases)})?.*?\b({'|'.join(pos_adj)})(?:{'|'.join(adj_postfix)})?\b.*?\b({'|'.join(neg_verb)})\b"


        # جستجو برای الگوهای مثبت در متن
        positive_matches1 = re.findall(pos_pattern1, text)
        positive_matches2 = re.findall(pos_pattern2, text)
        positive_matches3 = re.findall(pos_pattern3, text)
        positive_matches4 = re.findall(pos_pattern4, text)
        positive_matches5 = re.findall(pos_pattern5, text)
        # print(positive_matches1, positive_matches2, positive_matches3, positive_matches4, positive_matches5)
        # جستجو برای الگوهای منفی در متن
        negative_matches1 = re.findall(neg_pattern1, text)
        negative_matches2 = re.findall(neg_pattern2, text)
        negative_matches3 = re.findall(neg_pattern3, text)
        negative_matches4 = re.findall(neg_pattern4, text)
        negative_matches5 = re.findall(neg_pattern5, text)
        negative_matches6 = re.findall(neg_pattern6, text)

        # تعیین نتیجه بر اساس تطابق‌ها
        if positive_matches1 or positive_matches2 or positive_matches3 or positive_matches4 or positive_matches5:
            result = 'مثبت'
        elif negative_matches1 or negative_matches2 or negative_matches3 or negative_matches4 or negative_matches5 or negative_matches6:
            result = 'منفی'
        else:
          result = None

        return result, text


    def design(self, text):
      # Define your regex pattern with conjunction words and dots
      design_pattern = r"(زیبا|خوشگل|زشت|شیک|بدنه|آلومینیوم|ظاهر)"
      conjunction_words = r"\s+(و|اما|ولی|لیکن)\s+"
      split_pattern = f"{conjunction_words}|[.؟!]+"

      # Define lists of positive and negative words
      positive_words = ["خوب", "قشنگ", "جذاب", "شیک", "خوشگل", "زیبا"]
      negative_words = ["ضعیف", "مشکل دار", "مشکلات", "ناراضی", "عملکرد ضعیف" ,"بد", "ضعیف", "مشکل دار", "مشکلات", "بیخود", "بیریخت"]

      # Define a list of negative verbs
      negative_verbs = ["ندارد", "نداشت", "نداره", "نیست", "نیستش"]

      result = None
      if re.search(design_pattern, text):
        positive = any(word in sentence for word in positive_words)
        negative = any(word in sentence for word in negative_words)
        has_negative_verb = any(verb in sentence for verb in negative_verbs)

        if positive and not negative and not has_negative_verb:
            result = 'مثبت'
        elif not positive and negative:
            result = 'منفی'
        elif has_negative_verb:
            result = 'منفی'
        else:
            result = 'خنثی'

      return result, text


    def camera(self, text):
      # Define your regex pattern with conjunction words and dots
      camera_pattern = r"(عکس|تصویر|رزولوشن|مگاپیکسل|مگا پیکسل|لنز|شاتر|فوکوس|پرتره|دیافراگم|زوم|پیکسل|فلش|فلاش|اولترا واید| اولتراواید|پریسکوپ|تله فوتو|فوتو|فریم|دوربین|عکاسی|فیلم|فریم|سوژه|لرزشگیر|سلفی|فیلمبرداری|فیلم‌برداری|slow-motiom|slow motion|HDR|focus|resolution|pixel)"
      conjunction_words = r"\s+(و|اما|ولی|لیکن)\s+"
      split_pattern = f"{conjunction_words}|[.؟!]+"

      # Define lists of positive and negative words
      positive_words = ["خفن" ,"فوق‌العاده" ,"فوق العاده" ,"خوب", "قشنگ", "جذاب", "شیک", "خوشگل", "زیبا"]
      negative_words = ["ناراضی", "عملکرد ضعیف" ,"مزخرف" ,"چرت" ,"بد", "ضعیف", "مشکل دار", "مشکلات", "بیخود", "بیریخت"]

      # Define a list of negative verbs
      negative_verbs = ["ندارد", "نداشت", "نداره", "نیست", "نیستش"]


      result = None
      if re.search(camera_pattern, text):
        positive = any(word in sentence for word in positive_words)
        negative = any(word in sentence for word in negative_words)
        has_negative_verb = any(verb in sentence for verb in negative_verbs)

        if positive and not negative and not has_negative_verb:
            result = 'مثبت'
        elif not positive and negative:
            result = 'منفی'
        elif has_negative_verb:
            result = 'منفی'
        else:
            result = 'خنثی'

      return result, text

    def battery(self, text):
      # Define your regex pattern with conjunction words and dots
      battery_pattern = r"(باتری|باطری|شارژ|شارژر|باتری‌ها|باتریها)"
      conjunction_words = r"\s+(و|اما|ولی|لیکن)\s+"
      split_pattern = f"{conjunction_words}|[.؟!]+"

      # Define lists of positive and negative words
      positive_words = ["خوب", "عالی", "ضعیف", "قوی", "بدون مشکل", "عملکرد خوب"]
      negative_words = ["بد", "ضعیف", "مشکل دار", "مشکلات", "ناراضی", "عملکرد ضعیف"]

      # Define a list of negative verbs
      negative_verbs = ["ندارد", "نداشت", "نداره", "نیست", "نیستش"]

      result = None
      if re.search(battery_pattern, text):
        positive = any(word in sentence for word in positive_words)
        negative = any(word in sentence for word in negative_words)
        has_negative_verb = any(verb in sentence for verb in negative_verbs)

        if positive and not negative and not has_negative_verb:
            result = 'مثبت'
        elif not positive and negative:
            result = 'منفی'
        elif has_negative_verb:
            result = 'منفی'
        else:
            result = 'خنثی'

      return result, text

    def processor(self, text):
      # Define your regex pattern with conjunction words and dots
      processor_pattern = r"(پردازشگر|GPU|انویدیا|chipset|چیپست|core|پردازنده‌|چند هسته‌ای|CPU|اسنپ دراگون|مدیاتک|MIPS|X64|X86|اینتل|ARM|گرافیکی|تراشه|سی پی یو|Exynos|اگزینوس|Mediatek|اسنپدراگون|کوالکام|intel|qualcomm|snapdragon)"

      # Define lists of positive and negative words
      positive_words = ["خوب", "عالی", "ضعیف", "قوی", "بدون مشکل", "عملکرد خوب"]
      negative_words = ["بد", "ضعیف", "مشکل دار", "مشکلات", "ناراضی", "عملکرد ضعیف"]

      # Define a list of negative verbs
      negative_verbs = ["ندارد", "نداشت", "نداره", "نیست", "نیستش"]

      result = None
      if re.search(processor_pattern, text):
        positive = any(word in sentence for word in positive_words)
        negative = any(word in sentence for word in negative_words)
        has_negative_verb = any(verb in sentence for verb in negative_verbs)

        if positive and not negative and not has_negative_verb:
            result = 'مثبت'
        elif not positive and negative:
            result = 'منفی'
        elif has_negative_verb:
            result = 'منفی'
        else:
            result = 'خنثی'

      return result, text

    def size(self, text):
      # Define your regex pattern with conjunction words and dots
      battery_pattern = r"(سنگین|وزن|گرم|سبک|جیب|اینچ|بزرگ|کوچک|کوچیک|گنده|خوش دست)"

      # Define lists of positive and negative words
      positive_words = ["خوب", "عالی", "ضعیف", "قوی", "بدون مشکل", "عملکرد خوب"]
      negative_words = ["بد", "ضعیف", "مشکل دار", "مشکلات", "ناراضی", "عملکرد ضعیف"]

      # Define a list of negative verbs
      negative_verbs = ["ندارد", "نداشت", "نداره", "نیست", "نیستش"]

      result = None
      if re.search(battery_pattern, text):
        positive = any(word in sentence for word in positive_words)
        negative = any(word in sentence for word in negative_words)
        has_negative_verb = any(verb in sentence for verb in negative_verbs)

        if positive and not negative and not has_negative_verb:
            result = 'مثبت'
        elif not positive and negative:
            result = 'منفی'
        elif has_negative_verb:
            result = 'منفی'
        else:
            result = 'خنثی'

      return result, text

    def memory(self, text):
      # Define your regex pattern with conjunction words and dots
      memory_pattern = r"(رم|حافظه|گیگ|مموری|حافظه جانبی|حافظه داخلی|بایت|گیگا|مگا|ترا|فضا|SD|هارد|ظرفیت ذخیره سازی|استوریج|storage)"

      # Define lists of positive and negative words
      positive_words = ["خوب", "عالی", "ضعیف", "قوی", "بدون مشکل", "عملکرد خوب", 'زیاد', 'بالا', 'زیادی', 'بالایی', 'کافی']
      negative_words = ["بد", "ضعیف", "مشکل دار", "مشکلات", "ناراضی", "عملکرد ضعیف", 'کم', 'ناکافی', 'محدود']

      # Define a list of negative verbs
      negative_verbs = ["ندارد", "نداشت", "نداره", "نیست", "نیستش", ]

      result = None
      if re.search(memory_pattern, text):
        positive = any(word in sentence for word in positive_words)
        negative = any(word in sentence for word in negative_words)
        has_negative_verb = any(verb in sentence for verb in negative_verbs)

        if positive and not negative and not has_negative_verb:
            result = 'مثبت'
        elif not positive and negative:
            result = 'منفی'
        elif has_negative_verb:
            result = 'منفی'
        else:
            result = 'خنثی'

      return result, text

    def screen(self, text):
      # Define your regex pattern with conjunction words and dots

      # TODO --------------------


      screen_pattern = r"(رم|حافظه|گیگ|مموری|حافظه جانبی|حافظه داخلی|بایت|گیگا|مگا|ترا|فضا|SD|هارد|ظرفیت ذخیره سازی|استوریج|storage)"

      # TODO -----------------------

      # Define lists of positive and negative words
      positive_words = ["خوب", "عالی", "ضعیف", "قوی", "بدون مشکل", "عملکرد خوب", 'زیاد', 'بالا', 'زیادی', 'بالایی', 'کافی']
      negative_words = ["بد", "ضعیف", "مشکل دار", "مشکلات", "ناراضی", "عملکرد ضعیف", 'کم', 'ناکافی', 'محدود']

      # Define a list of negative verbs
      negative_verbs = ["ندارد", "نداشت", "نداره", "نیست", "نیستش", ]

      result = None
      if re.search(screen_pattern, text):
        positive = any(word in sentence for word in positive_words)
        negative = any(word in sentence for word in negative_words)
        has_negative_verb = any(verb in sentence for verb in negative_verbs)

        if positive and not negative and not has_negative_verb:
            result = 'مثبت'
        elif not positive and negative:
            result = 'منفی'
        elif has_negative_verb:
            result = 'منفی'
        else:
            result = 'خنثی'

      return result, text


    def run(self, text):
        sentences = persian_sentence_tokenizer_v3(text)
        result = {}

        result["نظر کلی"] = []
        for sent in sentences:
          general_opinion_res = self.general_opinion(sent)
          if general_opinion_res[0]:
              result["نظر کلی"].append({
                  "result": general_opinion_res[0]#TODO
                  , "sentence": general_opinion_res[1]#TODO
                  })

        result["طراحی"] = []
        for sent in sentences:
          design_res = self.design(sent)
          if design_res[0]:
              result["طراحی"].append({
                  "result": design_res[0]#TODO
                  , "sentence": design_res[1]#TODO
                  })

        result["دوربین"] = []
        for sent in sentences:
          camera_res = self.camera(sent)
          if camera_res[0]:
              result["دوربین"].append({
                  "result": camera_res[0]#TODO
                  , "sentence": camera_res[1]#TODO
                  })

        result["باطری"] = []
        for sent in sentences:
          battery_res = self.battery(sent)
          if battery_res[0]:
              result["باطری"].append({
                  "result": battery_res[0]#TODO
                  , "sentence": battery_res[1]#TODO
                  })

        result["پردازنده"] = []
        for sent in sentences:
          processor_res = self.processor(sent)
          if processor_res[0]:
              result["باطری"].append({
                  "result": processor_res[0]#TODO
                  , "sentence": processor_res[1]#TODO
                  })

        result["سایز"] = []
        for sent in sentences:
          size_res = self.size(sent)
          if size_res[0]:
              result["باطری"].append({
                  "result": size_res[0]#TODO
                  , "sentence": size_res[1]#TODO
                  })

        result["حافظه"] = []
        for sent in sentences:
          memory_res = self.memory(sent)
          if memory_res[0]:
              result["حافظه"].append({
                  "result": memory_res[0]#TODO
                  , "sentence": memory_res[1]#TODO
                  })

        result["صفحه نمایش"] = []
        for sent in sentences:
          screen_res = self.screen(sent)
          if screen_res[0]:
              result["صفحه نمایش"].append({
                  "result": screen_res[0]#TODO
                  , "sentence": screen_res[1]#TODO
                  })

        # feats = list(result.keys())
        # cols = ['دسته‌بندی', 'جمله']
        # print(feats)
        # print(result[feats[0]])
        # data_frame = pd.DataFrame(
        #                           np.array([[result[feat]['result'], result[feat]['sentence']] for feat in feats]),
        #                           columns=cols)


        return result




In [ ]:
import numpy as np
import pandas as pd

# **Initialize model and run**

In [ ]:
model = SmartPhoneFeatureExtractor()

In [ ]:
comments = [
    'گوشی فوق العاده ای نیست',
    'خوبی نیست',
    'موبایل بدی است',
    'گوشی خوبی هست',
    'در کل از گوشی راضی بودند ایش',
    'در کل خوب بود',
    'همه چیزش خوب بود',
    'این خوب نبود',
    'گوشی واقعا راضی کننده است. باطری گوشی یک روز پر کار رو همراهی میکنه و تو نیم ساعت 50 درصدش پر میشه دوربین فوق العاده ای داره و راضیتون میکنه پردازنده اش اصلا هنگ نمیکنه و داغ نمیشه و برنامه های سنگین رو راحت اجرا میکنه صفحه نمایشش با 900 نیت روشنایی کاملا تصاویر رو با کیفیت و واضع نشون میده. ظاهر کوشی هم خیلی شیک و خوشگله. رابط کاربری روان و حذابی داره. در کل گوشی قابل قبولی است و قیمتش نسبت به مشخصاتش خیلی ارزونه. در کل خوبه.'
]

In [ ]:
def process_comemnt(model, comment):
  normalized_text = model.sent_normalize(comment)
  print('input: ', normalized_text)
  res = model.run(normalized_text)
  print('output: ', )
  for feat in res.keys():
    print(f'{feat}: {res[feat]}')

In [ ]:
sents = persian_sentence_tokenizer_v3(comments[-1])
sents

['گوشی واقعا راضی کننده است.',
 'باطری گوشی یک روز پر کار رو همراهی میکنه',
 'و تو نیم ساعت 50 درصدش پر میشه دوربین فوق العاده ای داره',
 'و راضیتون میکنه',
 'پردازنده اش اصلا هنگ نمیکنه و و داغ نمیشه',
 'و برنامه های سنگین رو راحت اجرا میکنه',
 'صفحه نمایشش با 900 نیت روشنایی کاملا تصاویر رو با کیفیت و واضع نشون میده و .',
 'ظاهر کوشی هم خیلی شیک و خوشگله.',
 'رابط کاربری روان و حذابی داره.',
 'در کل گوشی قابل قبولی است و قیمتش نسبت به مشخصاتش خیلی ارزونه.',
 'در کل خوبه.']

In [ ]:
process_comemnt(model, comments[-1])


input:  گوشی واقعا راضی کنده است. باطری گوشی یک روز پر کار رو همراهی میکنه و تو نیم ساعت ۵۰ درصدش پر میشه دوربین فوق العاده ای داره و راضیتون میکنه پردازنده اش اصلا هنگ نمیکنه و داغ نمیشه و برنامه های سنگین رو راحت اجرا میکنه صفحه نمایشش با ۹۰۰ نیت روشنایی کاملا تصاویر رو با کیفیت و واضع نشون میده. ظاهر کوشی هم خیلی شیک و خوشگله. رابط کاربری روان و حذابی داره. در کل گوشی قابل قبولی است و قیمتش نسبت به مشخصاتش خیلی ارزونه. در کل خوبه.
output: 
نظر کلی: [{'result': 'مثبت', 'sentence': 'گوشی واقعا راضی کنده و است و .'}, {'result': 'مثبت', 'sentence': 'در کل گوشی قابل قبولی است و قیمتش نسبت به مشخصاتش خیلی ارزونه.'}, {'result': 'مثبت', 'sentence': 'در کل خوبه.'}]
طراحی: [{'result': 'خنثی', 'sentence': 'ظاهر کوشی هم خیلی شیک و خوشگله.'}]
دوربین: [{'result': 'مثبت', 'sentence': 'و تو نیم ساعت ۵۰ درصدش پر میشه دوربین فوق العاده ای داره'}]
باطری: [{'result': 'خنثی', 'sentence': 'باطری گوشی یک روز پر کار رو همراهی میکنه'}, {'result': 'خنثی', 'sentence': 'و برنامه های سنگین رو راحت اجرا میکنه'}]

In [ ]:
for com in comments:
  process_comemnt(model, com)

input:  گوشی فوق العاده ای نیست
output: 
نظر کلی: [{'result': 'منفی', 'sentence': 'گوشی فوق العاده ای نیست'}]
طراحی: []
دوربین: []
باطری: []
پردازنده: []
سایز: []
حافظه: []
صفحه نمایش: []
input:  خوبی نیست
output: 
نظر کلی: [{'result': 'منفی', 'sentence': 'خوبی نیست'}]
طراحی: []
دوربین: []
باطری: []
پردازنده: []
سایز: []
حافظه: []
صفحه نمایش: []
input:  موبایل بدی است
output: 
نظر کلی: [{'result': 'منفی', 'sentence': 'موبایل بدی است'}]
طراحی: []
دوربین: []
باطری: []
پردازنده: []
سایز: []
حافظه: []
صفحه نمایش: []
input:  گوشی خوبی هست
output: 
نظر کلی: [{'result': 'مثبت', 'sentence': 'گوشی خوبی هست'}]
طراحی: []
دوربین: []
باطری: []
پردازنده: []
سایز: []
حافظه: []
صفحه نمایش: []
input:  در کل از گوشی راضی بودند ایش
output: 
نظر کلی: [{'result': 'مثبت', 'sentence': 'در کل از گوشی راضی بودند ایش'}]
طراحی: []
دوربین: []
باطری: []
پردازنده: []
سایز: []
حافظه: []
صفحه نمایش: []
input:  در کل خوب بود
output: 
نظر کلی: [{'result': 'مثبت', 'sentence': 'در کل خوب بود'}]
طراحی: []
دوربین: []
باطری